# 求 ERC20 coin price

取得 WETH9 (WETH price) 合約的價格，

- https://www.oklink.com/zh-hant/ethereum/address/0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2/contract

為了得知真實價格，所以需要挑選美元穩定幣來比較 WETH，在區塊鏈中可以透過流動性合約為 WETH9 vs USDC/USDT 定價

首先查到 USDC 地址 https://www.oklink.com/zh-hant/ethereum/address/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48

再來查到流動性合約: https://www.oklink.com/zh-hant/ethereum/address/0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f

調用合約查詢 WETH9 vs USDC 的合約地址: 0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc

https://etherscan.io/address/0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f#readContract

https://etherscan.io/address/0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc

最終查詢合約可以得知當下價格

https://etherscan.io/address/0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc#readContract

ETH = 10853624529211/3692552607797834291386=**2.93932834e-9**

價格合理，小數點後面是單位問題，所以我們透過該合約的每一次成交時的轉換可以得知每個時刻的 ETH 價格

要抓的是 swap event

ETH price = data[0]/data[3] + data[2]/data[1]

### 從 GCP 讀數據

- data 就是事件的結果
- topic0 是 swap 換錢事件
- address 是上面查到的流動性合約地址

```sql
SELECT
  data, transaction_hash
FROM `bigquery-public-data.crypto_ethereum.logs`
WHERE
  address = LOWER('0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc')
  AND topics[SAFE_OFFSET(0)] = LOWER('0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822')
  AND (DATE(block_timestamp) >= '2025-11-30' AND DATE(block_timestamp) < '2025-12-1')
```

In [29]:
import pandas as pd
# pd.set_option('display.max_colwidth', None)

def hex_to_int(hex_str:str):
    # 检查字符串是否有效。如果只有 '0x' 或为空，则返回 NaN。
    if pd.isna(hex_str) or hex_str.lower() in ('0x', 'x'):
        return None  # 返回 None，Pandas 会将其识别为 NaN

    # 使用 int(string, 16) 将 Hex 字符串转换为整数
    try:
        # print(int("0x"+hex_str, 16))
        return int("0x"+hex_str, 16)
    except ValueError:
        # 如果 int() 转换失败（例如格式错误），也可以返回 NaN
        return None

df = pd.read_json("bquxjob_23ea0bcb_19b27cdf561.json", lines=False)
l_arr = [2+i*64 for i in range(5)]
df['input0'] = df['data'].str.slice(l_arr[0], l_arr[1])
df['input1'] = df['data'].str.slice(l_arr[1], l_arr[2])
df['output0'] = df['data'].str.slice(l_arr[2], l_arr[3])
df['output1'] = df['data'].str.slice(l_arr[3], l_arr[4])

df['tmp0'] = (df['input0'].apply(hex_to_int) / df['output1'].apply(hex_to_int)).fillna(0)
df['tmp1'] = (df['output0'].apply(hex_to_int) / df['input1'].apply(hex_to_int)).fillna(0)

df['USD2ETH'] = (df['tmp0'] + df['tmp1'])*10**12

print(df.head()['USD2ETH'])

0    2985.380544
1    3003.279204
2    2985.286587
3    3003.279303
4    3003.279344
Name: USD2ETH, dtype: float64
